<a href="https://colab.research.google.com/github/srilekhaj/Langchain-with-AI-agents/blob/main/CLTV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openpyxl pandas

In [2]:
import pandas as pd

df = pd.read_csv('/content/sample_data/Ecommerce_Data.csv')
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour
0,439570,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453,United Kingdom,04-11-2011,12
1,387281,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098,United Kingdom,10-10-2011,10
2,337863,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609,United Kingdom,13-09-2011,9
3,57628,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329,United Kingdom,14-01-2011,13
4,330897,565930,POST,POSTAGE,5,18.00,12685,France,08-09-2011,10


In [6]:
# prompt: Using dataframe df: calculate total revenue add in csv

# Calculate total revenue by multiplying Quantity and UnitPrice
df['TotalRevenue'] = (df['Quantity'] * df['UnitPrice']).round(2)

# Calculate the total revenue for all transactions
total_revenue = df['TotalRevenue'].sum().round(2)

# Print the total revenue
print(f"Total Revenue: {total_revenue}")


# Optionally, save the updated DataFrame with the 'TotalRevenue' column to a new CSV file
df.to_csv('data_with_revenue.csv', index=False)

Total Revenue: 1643139.2


In [7]:
# prompt: # Calculate total revenue by customer  and save it csv

# Group by CustomerID and calculate the total revenue for each customer
customer_revenue = df.groupby('CustomerID')['TotalRevenue'].sum().round(2)

# Convert the result to a DataFrame
customer_revenue = customer_revenue.reset_index()

# Rename the column for better clarity
customer_revenue = customer_revenue.rename(columns={'TotalRevenue': 'TotalRevenueByCustomer'})

# Save the customer revenue data to a CSV file
customer_revenue.to_csv('customer_revenue.csv', index=False)

In [8]:
# prompt: # Calculate total purchases per customer

# Group by CustomerID and calculate the total purchases for each customer
customer_purchases = df.groupby('CustomerID')['Quantity'].sum()

# Convert the result to a DataFrame
customer_purchases = customer_purchases.reset_index()

# Rename the column for better clarity
customer_purchases = customer_purchases.rename(columns={'Quantity': 'TotalPurchases'})

# Display the total purchases per customer
print(customer_purchases)

# Save the customer purchases data to a CSV file
customer_purchases.to_csv('customer_purchases.csv', index=False)

      CustomerID  TotalPurchases
0          12347             623
1          12348             531
2          12349             203
3          12350              36
4          12352             138
...          ...             ...
4106       18280               9
4107       18281              11
4108       18282              11
4109       18283             240
4110       18287             284

[4111 rows x 2 columns]


In [10]:
# prompt: can we calculate average purchase value and average purchase frequency  for each customer? is it possible

# Calculate average purchase value for each customer
average_purchase_value = df.groupby('CustomerID')['TotalRevenue'].mean().round(2)

# Calculate purchase frequency for each customer
purchase_frequency = df.groupby('CustomerID')['InvoiceNo'].nunique()

# Calculate average purchase frequency for each customer
average_purchase_frequency = purchase_frequency

# Combine the results into a single DataFrame
customer_metrics = pd.DataFrame({
    'AveragePurchaseValue': average_purchase_value,
    'AveragePurchaseFrequency': average_purchase_frequency
})

# Reset the index to make CustomerID a column
customer_metrics = customer_metrics.reset_index()

# Display the customer metrics
print(customer_metrics)

# Save the customer metrics data to a CSV file
customer_metrics.to_csv('customer_metrics.csv', index=False)

      CustomerID  AveragePurchaseValue  AveragePurchaseFrequency
0          12347                 23.89                         7
1          12348                 78.70                         4
2          12349                 25.40                         1
3          12350                 22.80                         1
4          12352                 17.05                         8
...          ...                   ...                       ...
4106       18280                 17.58                         1
4107       18281                 16.73                         1
4108       18282                 16.96                         2
4109       18283                  2.45                        14
4110       18287                 21.47                         2

[4111 rows x 3 columns]


In [13]:
# prompt: calculate customer lifespan date format 04-11-2011

import pandas as pd
from datetime import datetime

# Assuming your DataFrame 'df' has a 'InvoiceDate' column with dates in the format '04-11-2011'
# Convert 'InvoiceDate' to datetime objects
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Find the first purchase date for each customer
first_purchase_date = df.groupby('CustomerID')['Date'].min()

# Find the last purchase date for each customer
last_purchase_date = df.groupby('CustomerID')['Date'].max()

# Calculate customer lifespan (in days)
customer_lifespan_days = (last_purchase_date - first_purchase_date).dt.days

# Create a new DataFrame with customer lifespan in days
customer_lifespan = pd.DataFrame({'CustomerLifespanDays': customer_lifespan_days})

# Reset the index to make CustomerID a column
customer_lifespan = customer_lifespan.reset_index()

# Display the customer lifespan
print(customer_lifespan)

# Optionally, save the customer lifespan data to a CSV file
customer_lifespan.to_csv('customer_lifespan.csv', index=False)

# You can also calculate the lifespan in other formats (e.g., months, years) if needed.
# For example, to calculate lifespan in months:
# customer_lifespan_months = ((last_purchase_date - first_purchase_date) / np.timedelta64(1, 'M')).round(2)

      CustomerID  CustomerLifespanDays
0          12347                   365
1          12348                   283
2          12349                     0
3          12350                     0
4          12352                   260
...          ...                   ...
4106       18280                     0
4107       18281                     0
4108       18282                   119
4109       18283                   334
4110       18287                   143

[4111 rows x 2 columns]


In [16]:
# prompt: calculate the cltv value

# Merge customer_revenue, customer_metrics, and customer_lifespan DataFrames
cltv_df = pd.merge(customer_revenue, customer_metrics, on='CustomerID', how='left')
cltv_df = pd.merge(cltv_df, customer_lifespan, on='CustomerID', how='left')

# Calculate CLTV using the formula: CLTV = (Average Purchase Value * Average Purchase Frequency) * Customer Lifespan
cltv_df['CLTV'] = (cltv_df['AveragePurchaseValue'] * cltv_df['AveragePurchaseFrequency']) * cltv_df['CustomerLifespanDays']
cltv_df['CLTV'] = cltv_df['CLTV'].round(2)
cltv_df['CLTV']

# Display the CLTV for each customer
print(cltv_df)

# Optionally, save the CLTV data to a CSV file
cltv_df.to_csv('customer_cltv.csv', index=False)

      CustomerID  TotalRevenueByCustomer  AveragePurchaseValue  \
0          12347                 1146.59                 23.89   
1          12348                  550.92                 78.70   
2          12349                  406.32                 25.40   
3          12350                   45.60                 22.80   
4          12352                  426.30                 17.05   
...          ...                     ...                   ...   
4106       18280                   52.75                 17.58   
4107       18281                   33.45                 16.73   
4108       18282                   67.85                 16.96   
4109       18283                  325.54                  2.45   
4110       18287                  364.96                 21.47   

      AveragePurchaseFrequency  CustomerLifespanDays      CLTV  
0                            7                   365  61038.95  
1                            4                   283  89088.40  
2           